In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/isear-dataset/eng_dataset.csv


In [2]:
data=pd.read_csv('/kaggle/input/isear-dataset/eng_dataset.csv')

In [3]:
data

,ID,sentiment,content
0,10941,anger,At the point today where if someone says somet...
1,10942,anger,@CorningFootball IT'S GAME DAY!!!! T MIN...
2,10943,anger,This game has pissed me off more than any othe...
3,10944,anger,@spamvicious I've just found out it's Candice ...
4,10945,anger,@moocowward @mrsajhargreaves @Melly77 @GaryBar...
...,...,...,...
7097,40781,sadness,@VivienLloyd Thank you so much! Just home - st...
7098,40782,sadness,Just put the winter duvet on ☃️❄️🌬☔️
7099,40783,sadness,@SilkInSide @TommyJoeRatliff that's so pretty!...
7100,40784,sadness,@BluesfestByron second artist announcement loo...


In [24]:
data['sentiment'].unique()

array(['anger', 'fear', 'joy', 'sadness'], dtype=object)

In [33]:
for i in data['sentiment'].unique():
    print("number of rows with '",i,"' sentiment is: ",data[data['sentiment']==i].sentiment.count())
    
print("-"*80)
print("sum of all sentiments is: ",data['sentiment'].count())

number of rows with ' anger ' sentiment is:  1701
number of rows with ' fear ' sentiment is:  2252
number of rows with ' joy ' sentiment is:  1616
number of rows with ' sadness ' sentiment is:  1533
--------------------------------------------------------------------------------
sum of all sentiments is:  7102


In [34]:
sentences=data['content']
sentiment=data['sentiment']

In [59]:
sentences=sentences.apply(lambda x:x.lower())

In [38]:
from string import punctuation

In [37]:
sentences

0       at the point today where if someone says somet...
1       @corningfootball  it's game day!!!!      t min...
2       this game has pissed me off more than any othe...
3       @spamvicious i've just found out it's candice ...
4       @moocowward @mrsajhargreaves @melly77 @garybar...
                              ...                        
7097    @vivienlloyd thank you so much! just home - st...
7098                just put the winter duvet on ☃️❄️🌬☔️ 
7099    @silkinside @tommyjoeratliff that's so pretty!...
7100    @bluesfestbyron second artist announcement loo...
7101    i can literally eat creamy pesto pasta topped ...
Name: content, Length: 7102, dtype: object

In [42]:
from collections import Counter

## Build a dictionary that maps words to integers
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

## use the dict to tokenize each review in reviews_split
## store the tokenized reviews in reviews_ints
reviews_ints = []
for review in reviews_split:
    reviews_ints.append([vocab_to_int[word] for word in review.split()])

In [53]:
sentiment=sentiment.map({'anger':0,'sadness':1,'fear':2,'joy':3})

In [54]:
sentiment.unique()

array([0, 2, 3, 1])

In [133]:
from sklearn.preprocessing import OneHotEncoder

In [134]:
enc = OneHotEncoder(handle_unknown='ignore')
Y = enc.fit_transform(np.array(sentiment).reshape(-1,1)).toarray()
print(Y.shape)

(7102, 4)


In [136]:
sentiment

0       0
1       0
2       0
3       0
4       0
       ..
7097    1
7098    1
7099    1
7100    1
7101    1
Name: sentiment, Length: 7102, dtype: int64

In [135]:
Y

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.]])

In [62]:
import tensorflow as tf

In [66]:
sentences

0       at the point today where if someone says somet...
1       @corningfootball  it's game day!!!!      t min...
2       this game has pissed me off more than any othe...
3       @spamvicious i've just found out it's candice ...
4       @moocowward @mrsajhargreaves @melly77 @garybar...
                              ...                        
7097    @vivienlloyd thank you so much! just home - st...
7098                just put the winter duvet on ☃️❄️🌬☔️ 
7099    @silkinside @tommyjoeratliff that's so pretty!...
7100    @bluesfestbyron second artist announcement loo...
7101    i can literally eat creamy pesto pasta topped ...
Name: content, Length: 7102, dtype: object

In [81]:
from keras.models import Sequential

In [87]:
!kaggle datasets download -d watts2/glove6b50dtxt

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Dataset URL: https://www.kaggle.com/datasets/watts2/glove6b50dtxt
License(s): CC0-1.0
 87%|██████████████████████████████████     | 59.0M/67.7M [00:00<00:00, 153MB/s]
100%|███████████████████████████████████████| 67.7M/67.7M [00:00<00:00, 159MB/s]


In [88]:
!unzip glove6b50dtxt.zip

Archive:  glove6b50dtxt.zip
  inflating: glove.6B.50d.txt        


In [89]:
glove='/kaggle/working/glove.6B.50d.txt'

In [90]:
def load_glove_embeddings(path):
    embeddings_index = {}
    with open(path, 'r', encoding='utf8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

Glove = load_glove_embeddings(glove)

In [91]:
def preprocess(Sentences):
    sentences = tf.strings.substr(Sentences, 0, 300)
    sentences = tf.strings.regex_replace(sentences, b"<br\\s*/?>", b" ")
    sentences = tf.strings.regex_replace(sentences, b"[^a-zA-Z']", b" ")
    sentences = tf.strings.split(sentences)
    sentences = tf.strings.lower(sentences)
    sentences = sentences.to_tensor(default_value=b"<pad>")
    return sentences

In [93]:
sentences=preprocess(sentences)

In [94]:
sentences

<tf.Tensor: shape=(7102, 34), dtype=string, numpy=
array([[b'at', b'the', b'point', ..., b'<pad>', b'<pad>', b'<pad>'],
       [b'corningfootball', b"it's", b'game', ..., b'<pad>', b'<pad>',
        b'<pad>'],
       [b'this', b'game', b'has', ..., b'<pad>', b'<pad>', b'<pad>'],
       ...,
       [b'silkinside', b'tommyjoeratliff', b"that's", ..., b'<pad>',
        b'<pad>', b'<pad>'],
       [b'bluesfestbyron', b'second', b'artist', ..., b'<pad>', b'<pad>',
        b'<pad>'],
       [b'i', b'can', b'literally', ..., b'<pad>', b'<pad>', b'<pad>']],
      dtype=object)>

In [95]:
def encoding(sentences, Glove):
    Encoded_vec = []
    for sentence in sentences:
        sent_vec = []
        for token in sentence:
            token = token.numpy().decode('utf-8')
            if token in Glove:
                sent_vec.append(Glove[token])
            else:
                sent_vec.append(np.zeros(50))
        Encoded_vec.append(sent_vec)
    return Encoded_vec

In [96]:
encoded=encoding(sentences,Glove)
sentences_X=np.array(encoded)

In [97]:
sentences_X.shape

(7102, 34, 50)

In [98]:
from sklearn.model_selection import train_test_split

In [121]:
sentiment.shape

(7102,)

In [147]:
X_train,X_test,Y_train,Y_test=train_test_split(sentences_X,Y,test_size=0.2,shuffle=True,random_state=42)

In [103]:
Y_test

6095    3
4447    2
4677    2
4832    2
3796    0
       ..
6522    1
3829    0
1649    2
4819    2
1720    2
Name: sentiment, Length: 1421, dtype: int64

# Pytorch

In [200]:
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(Y_train))
# valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(Y_test))

# dataloaders
batch_size = 50

# make sure the SHUFFLE your training data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
# valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [201]:
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size=50, hidden_size=100, batch_first=True)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(100, 4)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        lstm_out = lstm_out[:, -1, :]  # Get the last output from the LSTM
        x = self.dropout(lstm_out)
        x = self.fc(x)
        x = self.softmax(x)
        return x

In [202]:
model_pytorch = LSTMModel()

In [203]:
lr=0.001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model_pytorch.parameters(), lr=lr)

In [216]:
model_pytorch

LSTMModel(
  (lstm): LSTM(50, 100, batch_first=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=100, out_features=4, bias=True)
  (softmax): Softmax(dim=1)
)

In [ ]:
output=model_pytorch()

In [221]:
import torch
import torch.nn as nn

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_pytorch.to(device)

# Training parameters
epochs = 10

# Training loop
for epoch in range(epochs):
    model_pytorch.train()  # Set the model to training mode
    train_loss = 0.0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Reshape inputs if necessary
        inputs = inputs.view(inputs.size(0), -1, 50)  # Ensure the shape is [batch_size, sequence_length, input_size]

        optimizer.zero_grad()  # Clear the gradients

        # Forward pass
        outputs = model_pytorch(inputs.float())
        
        # Compute loss
        loss = criterion(outputs, labels.float())
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Accumulate loss
        train_loss += loss.item()
    
    # Average loss for the epoch
    train_loss /= len(train_loader)
    
    print(f'Epoch {epoch+1}/{epochs}, Loss: {train_loss:.4f}')
    
# Testing loop
model_pytorch.eval()  # Set the model to evaluation mode
test_loss = 0.0

with torch.no_grad():  # Disable gradient calculation
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Reshape inputs if necessary
        inputs = inputs.view(inputs.size(0), -1, 50)  # Ensure the shape is [batch_size, sequence_length, input_size]

        # Forward pass
        outputs = model_pytorch(inputs.float())
        
        # Compute loss
        loss = criterion(outputs, labels.float())
        
        # Accumulate loss
        test_loss += loss.item()

# Average loss for the test set
test_loss /= len(test_loader)
print(f'Test Loss: {test_loss:.4f}')


RuntimeError: Invalid argument

# Tensorflow

In [148]:
model_tf=Sequential()

In [112]:
from keras.layers import LSTM,Dropout,Dense

In [149]:
model_tf.add(LSTM(100, input_shape=(100, 50)))

/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [150]:
model_tf.add(Dropout(0.2))
model_tf.add(Dense(4,activation='softmax'))

In [151]:
model_tf.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [152]:
model_tf.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 100)            │        60,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │           404 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 60,804 (237.52 KB)

 Trainable params: 60,804 (237.52 KB)

 Non-trainable params: 0 (0.00 B)

In [153]:
model_tf.fit(X_train,Y_train,epochs=20, batch_size=64)

Epoch 1/20
89/89 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.3211 - loss: 1.3595
Epoch 2/20
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4348 - loss: 1.2274
Epoch 3/20
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5447 - loss: 1.0596
Epoch 4/20
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5883 - loss: 0.9883
Epoch 5/20
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6404 - loss: 0.9094
Epoch 6/20
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6858 - loss: 0.8084
Epoch 7/20
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6958 - loss: 0.7763
Epoch 8/20
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7482 - loss: 0.6739
Epoch 9/20
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7699 - loss: 0.6163
Epoch 10/20
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7898 - loss: 0.5548
Epoch 11/20
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8033 - loss: 0.5405
Epoch 12/20
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8320 - lo

In [155]:
model_tf.fit(X_train,Y_train,epochs=20, batch_size=64)

Epoch 1/20
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8982 - loss: 0.2703
Epoch 2/20
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9015 - loss: 0.2693
Epoch 3/20
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9014 - loss: 0.2799
Epoch 4/20
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9000 - loss: 0.2748
Epoch 5/20
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9002 - loss: 0.2893
Epoch 6/20
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9102 - loss: 0.2422
Epoch 7/20
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9261 - loss: 0.2028
Epoch 8/20
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9275 - loss: 0.1920
Epoch 9/20
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9300 - loss: 0.1859
Epoch 10/20
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9260 - loss: 0.2103
Epoch 11/20
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9407 - loss: 0.1633
Epoch 12/20
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9342 - lo

In [158]:
loss,acc=model_tf.evaluate(X_test, Y_test, batch_size=64)

23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7694 - loss: 0.9485


In [159]:
loss

0.9740169644355774

In [160]:
acc

0.7741027474403381

# Prediction

In [177]:
map={0:'anger',1:'fear',2:'joy',3:'sadness'}

In [186]:
def predict_tf(sentence):
    sentence=preprocess(sentence)
    sentence=encoding(sentence,Glove)
    sentence=np.array(sentence)
    return map[np.argmax(model_tf.predict(sentence))]


In [193]:
twt = ["seif"]

In [194]:
predict_tf(twt)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step


'joy'